In [1]:
!pip install gradio
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# --- Sample Cricket Match Data (REPLACE WITH YOUR ACTUAL DATA) ---
data = {
    'TeamA': ['India', 'Australia', 'England', 'India', 'South Africa', 'Australia', 'England', 'India', 'New Zealand', 'Australia'],
    'TeamB': ['Australia', 'England', 'India', 'South Africa', 'England', 'India', 'Australia', 'New Zealand', 'England', 'South Africa'],
    'Venue': ['Home', 'Away', 'Neutral', 'Home', 'Away', 'Neutral', 'Home', 'Away', 'Neutral', 'Home'],
    'Toss_Won_A': [1, 0, 1, 1, 0, 1, 0, 1, 0, 1],  # 1 if Team A won the toss, 0 otherwise
    'TeamA_Rank': [1, 2, 3, 1, 4, 2, 3, 1, 5, 2],
    'TeamB_Rank': [2, 3, 1, 4, 3, 1, 2, 5, 3, 4],
    'Head_to_Head_Wins_A': [3, 2, 1, 4, 0, 2, 1, 5, 1, 3],
    'Recent_Form_A': [0.8, 0.6, 0.7, 0.9, 0.5, 0.75, 0.65, 0.92, 0.58, 0.88],  # Win percentage in last 5 matches
    'Recent_Form_B': [0.7, 0.55, 0.8, 0.6, 0.75, 0.88, 0.5, 0.65, 0.72, 0.6],
    'Win_A': [1, 0, 0, 1, 0, 1, 1, 1, 0, 1]  # 1 if Team A won, 0 if Team B won
}
df = pd.DataFrame(data)

# --- Data Preprocessing ---
label_encoders = {}
for column in ['TeamA', 'TeamB', 'Venue']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Separate features (X) and target (y)
X = df[['TeamA', 'TeamB', 'Venue', 'Toss_Won_A', 'TeamA_Rank', 'TeamB_Rank', 'Head_to_Head_Wins_A', 'Recent_Form_A', 'Recent_Form_B']]
y = df['Win_A']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
numerical_features = ['TeamA_Rank', 'TeamB_Rank', 'Head_to_Head_Wins_A', 'Recent_Form_A', 'Recent_Form_B']
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# --- Train the Logistic Regression model ---
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# --- Prediction Function for Gradio ---
def predict_winner(team_a, team_b, venue, toss_won_a, team_a_rank, team_b_rank, head_to_head_wins_a, recent_form_a, recent_form_b):
    try:
        # Encode categorical inputs
        team_a_encoded = label_encoders['TeamA'].transform([team_a])[0]
        team_b_encoded = label_encoders['TeamB'].transform([team_b])[0]
        venue_encoded = label_encoders['Venue'].transform([venue])[0]

        input_data = pd.DataFrame({
            'TeamA': [team_a_encoded],
            'TeamB': [team_b_encoded],
            'Venue': [venue_encoded],
            'Toss_Won_A': [toss_won_a],
            'TeamA_Rank': [team_a_rank],
            'TeamB_Rank': [team_b_rank],
            'Head_to_Head_Wins_A': [head_to_head_wins_a],
            'Recent_Form_A': [recent_form_a],
            'Recent_Form_B': [recent_form_b]
        })

        # Scale numerical inputs
        numerical_features = ['TeamA_Rank', 'TeamB_Rank', 'Head_to_Head_Wins_A', 'Recent_Form_A', 'Recent_Form_B']
        input_data[numerical_features] = scaler.transform(input_data[numerical_features])

        probability = model.predict_proba(input_data)[0][1]  # Probability of Team A winning
        prediction = f"{team_a} is likely to win" if probability >= 0.5 else f"{team_b} is likely to win"
        confidence = f"{probability * 100:.2f}% probability for {team_a} to win."
        return prediction, confidence
    except Exception as e:
        return f"Error during prediction: {e}", ""

# --- Gradio Interface ---
iface = gr.Interface(
    fn=predict_winner,
    inputs=[
        gr.Dropdown(choices=list(label_encoders['TeamA'].classes_), label="Team A"),
        gr.Dropdown(choices=list(label_encoders['TeamB'].classes_), label="Team B"),
        gr.Dropdown(choices=list(label_encoders['Venue'].classes_), label="Venue"),
        gr.Radio([0, 1], label="Did Team A win the toss? (1=Yes, 0=No)"),
        gr.Number(label="Team A Rank (Lower is better)"),
        gr.Number(label="Team B Rank (Lower is better)"),
        gr.Number(label="Head-to-Head Wins for Team A"),
        gr.Slider(0, 1, step=0.01, label="Recent Form of Team A (Win Percentage)"),
        gr.Slider(0, 1, step=0.01, label="Recent Form of Team B (Win Percentage)")
    ],
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Confidence")
    ],
    title="Cricket Match Winner Predictor (Logistic Regression)",
    description="Predict the likely winner of a cricket match based on team statistics and match conditions."
)

iface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.5 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aec32e4a559ebcc4ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
